# Grid Search

В прошлом пункте у нас получилось сделать knn, теперь давайте напишем для него grid search:

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns; sns.set_style()

import utils
from knn import knnRegression

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
X = pd.read_csv('data/processed/features.csv').to_numpy()
y = pd.read_csv('data/processed/labels.csv')['class'].to_numpy()

Схема пайплайна будет такая: сначала мы энкодим входные `y`-и в вектора, потом применяем `knn`, потом декодим обратно.

Поэтому энкодинг мы сделаем очень неаккуратно:

In [3]:
from __future__ import annotations

class OneHotKnnClassifier(utils.BasicLayer):
    def __init__(self):
        super().__init__()
        self.knn = knnRegression()
    
    def _push_attributes(self):
        for attribute in ['kernel', 'metric', 'window', 'window_parameter']:
            setattr(self.knn, attribute, getattr(self, attribute))
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> OneHotKnnClassifier:
        self._push_attributes()
        
        y_enc = np.zeros([y.shape[0], 3])
        
        for i, clazz in enumerate(y):
            y_enc[i][clazz - 1] = 1.
        
        self.knn.fit(X, y_enc)
        
        return self
    
    def transform(self, X: np.ndarray) -> np.ndarray:
        res = self.knn.transform(X)
        
        return np.argmax(res, axis=1) + 1

Ух ну и грязища. Давайте прогоним это:

In [4]:
sample_features = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]).astype('float')

sample_labels = np.array([1, 2, 2, 3])


clz = OneHotKnnClassifier()
setattr(clz, 'kernel', 'uniform')
setattr(clz, 'metric', 'euclid')
setattr(clz, 'window', 'variadic')
setattr(clz, 'window_parameter', 1)

clz.fit_transform(sample_features, sample_labels)

array([1, 2, 2, 3])

In [5]:
sample_test_features = np.array([
  [.5, .5],
  [.5, 1.]
])

clz.transform(sample_test_features)

array([3, 3])

Кажется, наша обёртка работает. Поехали пилить гридсёрч.

In [6]:
pipeline = utils.Pipeline([
    ('knn', OneHotKnnClassifier())
])

Из-за разного значения параметра window_parameter приходится жить так:

In [7]:
gs_variadic = utils.GridSearch(pipeline, {
    ('knn', 'kernel'): ['uniform', 'triangular', 'epanechnikov', 'quartic'],
    ('knn', 'metric'): ['euclid', 'manhattan', 'chebyshev'],
    ('knn', 'window'): ['variadic'],
    ('knn', 'window_parameter'): range(1, 15)
})

In [8]:
gs_variadic.fit(X, y)

(('uniform', 'euclid', 'variadic', 1), 0.365503739715457)
(('uniform', 'euclid', 'variadic', 2), 0.32000274857417715)
(('uniform', 'euclid', 'variadic', 3), 0.35182381907272736)
(('uniform', 'euclid', 'variadic', 4), 0.3229315905427134)
(('uniform', 'euclid', 'variadic', 5), 0.3398489079063471)
(('uniform', 'euclid', 'variadic', 6), 0.3333266919764088)
(('uniform', 'euclid', 'variadic', 7), 0.3271443078771854)
(('uniform', 'euclid', 'variadic', 8), 0.329007948573166)
(('uniform', 'euclid', 'variadic', 9), 0.34784275467423603)
(('uniform', 'euclid', 'variadic', 10), 0.3305643820859285)
(('uniform', 'euclid', 'variadic', 11), 0.3468315317586697)


KeyboardInterrupt: 

In [ ]:
gs_variadic.best_score, gs_variadic.best_params

Аналогичный гридсёрч можно написать и для `fixed`, но у меня беды со временем. Поэтому в следующем ноутбуке я просто нарисую графики F-меры чтобы было ясно что я умею.

In [ ]:
gs_fixed = utils.GridSearch(pipeline, {
    ('knn', 'kernel'): ['uniform', 'triangular', 'epanechnikov', 'quartic'],
    ('knn', 'metric'): ['euclid', 'manhattan', 'chebyshev'],
    ('knn', 'window'): ['fixed'],
    ('knn', 'window_parameter'): np.linspace(0, 100, num=20)
})

In [ ]:
gs_fixed.fit(X, y)

In [ ]:
gs_fixed.best_score, gs_fixed.best_params

In [ ]:
scores = [score for entry, score in gs_variadic.history if entry[0] == 'quartic' and entry[1] == 'manhattan']

plt.plot(scores)